In [205]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [206]:
df=pd.read_csv(f'dataset\GUIDE_Test\GUIDE_Test.csv',low_memory=False)
print(len(df))

1048575


In [207]:
result=df

In [208]:
def check_missing_values(df1, threshold_percentage=0.2):
    columns_to_drop = set()

    nan_counts = df1.isnull().mean()
    nan_columns = nan_counts[nan_counts > threshold_percentage]
    for col, percent in nan_columns.items():
        print(f"Column '{col}' has {percent:.2%} missing values.")
        columns_to_drop.add(col)

    empty_str_count = (df1 == '').mean()
    empty_str_columns = empty_str_count[empty_str_count > threshold_percentage]
    for col, percent in empty_str_columns.items():
        print(f"Column '{col}' has {percent:.2%} empty string values.")
        columns_to_drop.add(col)

    space_only_count = (df1.applymap(lambda x: isinstance(x, str) and x.isspace())).mean()
    space_only_columns = space_only_count[space_only_count > threshold_percentage]
    for col, percent in space_only_columns.items():
        print(f"Column '{col}' has {percent:.2%} space-only values.")
        columns_to_drop.add(col)

    problematic_values = ['N/A', 'NA', 'null', 'NULL']
    for value in problematic_values:
        problematic_count = (df1 == value).mean()
        problematic_columns = problematic_count[problematic_count > threshold_percentage]
        for col, percent in problematic_columns.items():
            print(f"Column '{col}' has {percent:.2%} '{value}' values.")
            columns_to_drop.add(col)

    return df1
result = check_missing_values(result)


Column 'MitreTechniques' has 55.56% missing values.
Column 'ActionGrouped' has 99.95% missing values.
Column 'ActionGranular' has 99.95% missing values.
Column 'EmailClusterId' has 98.99% missing values.
Column 'ThreatFamily' has 99.25% missing values.
Column 'ResourceType' has 99.93% missing values.
Column 'Roles' has 97.36% missing values.
Column 'AntispamDirection' has 98.15% missing values.
Column 'SuspicionLevel' has 84.37% missing values.
Column 'LastVerdict' has 76.09% missing values.


C:\Users\Admin\AppData\Local\Temp\ipykernel_18196\2916278563.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  space_only_count = (df1.applymap(lambda x: isinstance(x, str) and x.isspace())).mean()


In [209]:
df['LastVerdict']=df['LastVerdict'].fillna('No Verdict')

In [210]:
print((df['LastVerdict'] == 'DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66').sum())
print((df['LastVerdict'] == 'DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2').sum())

12
0


In [211]:
df.drop(columns=['Usage','ActionGrouped','ActionGranular','EmailClusterId','ThreatFamily','AntispamDirection','SuspicionLevel','Roles','ResourceType'],inplace=True)

In [212]:
df.drop(columns=['Timestamp','MitreTechniques'],inplace=True)

In [213]:
df['Category'].unique()

array(['LateralMovement', 'CommandAndControl', 'InitialAccess',
       'Discovery', 'SuspiciousActivity', 'Impact', 'CredentialAccess',
       'Exfiltration', 'UnwantedSoftware', 'DefenseEvasion', 'Malware',
       'Execution', 'Persistence', 'CredentialStealing', 'Collection',
       'Ransomware', 'Exploit', 'PrivilegeEscalation', 'WebExploit'],
      dtype=object)

In [214]:
print(df['EntityType'].unique())

['User' 'Machine' 'Process' 'CloudLogonSession' 'Ip' 'File' 'MailMessage'
 'CloudLogonRequest' 'Url' 'CloudApplication' 'MailCluster' 'Mailbox'
 'RegistryValue' 'AzureResource' 'GenericEntity' 'Malware' 'RegistryKey'
 'OAuthApplication' 'Blob' 'MailboxConfiguration' 'SecurityGroup'
 'BlobContainer' 'ActiveDirectoryDomain' 'Nic' 'IoTDevice']


In [215]:
df['EvidenceRole'].unique()

array(['Impacted', 'Related'], dtype=object)

In [216]:
df['LastVerdict'].unique()

array(['Suspicious', 'No Verdict', 'Malicious', 'NoThreatsFound',
       'DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66'], dtype=object)

In [217]:
valid_verdicts = ['No Verdict', 'Suspicious', 'Malicious', 'NoThreatsFound']
df = df[df['LastVerdict'].isin(valid_verdicts)]

In [218]:
df = df.dropna()
df.reset_index(drop=True, inplace=True)

In [219]:
columns_to_encode = ['Category', 'EntityType', 'EvidenceRole', 'LastVerdict', 'IncidentGrade']

for column in columns_to_encode:
    label_encoder = joblib.load(f'models/{column}_label_encoder.pkl')
    
    df[column] = label_encoder.transform(df[column])

In [220]:
columns_to_scale = ['CountryCode', 'State', 'City']
scaled_data_new =df[columns_to_scale]
pca = joblib.load('models/location.pkl')
df['location'] = pca.transform(scaled_data_new)

In [221]:
columns_to_scale = ['AccountSid', 'AccountUpn', 'AccountObjectId','AccountName']
scaled_data_new =df[columns_to_scale]
pca = joblib.load('models/Account.pkl')
df['Account'] = pca.transform(scaled_data_new)

In [222]:
columns_to_scale = ['RegistryValueName', 'RegistryValueData']
scaled_data_new =df[columns_to_scale]
pca = joblib.load('models/RegistryName.pkl')
df['RegistryName'] = pca.transform(scaled_data_new)

In [223]:
columns_to_scale = ['FileName','FolderPath','Sha256']
scaled_data_new =df[columns_to_scale]
pca = joblib.load('models/Path.pkl')
df['Path'] = pca.transform(scaled_data_new)

In [224]:
df=df.drop(['CountryCode', 'State', 'City','OSVersion','ApplicationName','AccountSid', 'AccountUpn','Sha256', 'AccountObjectId','AccountName','RegistryValueName', 'RegistryValueData','FileName','FolderPath'],axis=1)

In [225]:
df.reset_index(drop=True,inplace=True)
len(df)

1048563

In [226]:
X_test_1 = df.drop('IncidentGrade', axis=1) 
y_test = df['IncidentGrade']

In [227]:
scaler = joblib.load('models/scaler.pkl')
X_test= scaler.transform(X_test_1)

In [228]:
df1=pd.read_csv('train.csv')

In [229]:
X_train = df1.drop('IncidentGrade', axis=1) 
y_train = df1['IncidentGrade']

In [230]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


0.5812697949479431
              precision    recall  f1-score   support

           0       0.64      0.56      0.60    443490
           1       0.40      0.42      0.41    228276
           2       0.62      0.71      0.66    376797

    accuracy                           0.58   1048563
   macro avg       0.56      0.56      0.56   1048563
weighted avg       0.58      0.58      0.58   1048563



In [231]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_test)
print(accuracy_score(y_test, y_pred_lda))
print(classification_report(y_test, y_pred_lda))


c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(


0.5747599333564125
              precision    recall  f1-score   support

           0       0.64      0.55      0.59    443490
           1       0.37      0.45      0.41    228276
           2       0.65      0.67      0.66    376797

    accuracy                           0.57   1048563
   macro avg       0.56      0.56      0.56   1048563
weighted avg       0.59      0.57      0.58   1048563



In [232]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier()
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
print(accuracy_score(y_test, y_pred_ridge))
print(classification_report(y_test, y_pred_ridge))



c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeClassifier was fitted with feature names
  warnings.warn(


0.5706085375890624
              precision    recall  f1-score   support

           0       0.65      0.53      0.59    443490
           1       0.40      0.39      0.40    228276
           2       0.59      0.72      0.65    376797

    accuracy                           0.57   1048563
   macro avg       0.55      0.55      0.54   1048563
weighted avg       0.57      0.57      0.57   1048563



In [235]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print(accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.8982912805429908
              precision    recall  f1-score   support

           0       0.91      0.92      0.91    443490
           1       0.83      0.84      0.83    228276
           2       0.93      0.91      0.92    376797

    accuracy                           0.90   1048563
   macro avg       0.89      0.89      0.89   1048563
weighted avg       0.90      0.90      0.90   1048563



In [236]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_test)
print(accuracy_score(y_test, y_pred_ada))
print(classification_report(y_test, y_pred_ada))


c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


0.7284855559465668
              precision    recall  f1-score   support

           0       0.72      0.77      0.75    443490
           1       0.60      0.66      0.63    228276
           2       0.83      0.72      0.77    376797

    accuracy                           0.73   1048563
   macro avg       0.72      0.72      0.72   1048563
weighted avg       0.74      0.73      0.73   1048563



In [237]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print(accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


0.8830818939825266
              precision    recall  f1-score   support

           0       0.92      0.87      0.89    443490
           1       0.76      0.89      0.82    228276
           2       0.93      0.89      0.91    376797

    accuracy                           0.88   1048563
   macro avg       0.87      0.88      0.88   1048563
weighted avg       0.89      0.88      0.88   1048563



In [238]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
print(accuracy_score(y_test, y_pred_lgb))
print(classification_report(y_test, y_pred_lgb))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.270367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3039
[LightGBM] [Info] Number of data points in the train set: 6095859, number of used features: 24
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
0.8675520688790278
              precision    recall  f1-score   support

           0       0.90      0.86      0.88    443490
           1       0.74      0.89      0.81    228276
           2       0.93      0.86      0.90    376797

    accuracy                           0.87   1048563
   macro avg       0.86      0.87      0.86   1048563
weighted avg       0.88      0.87      0.87   1048563



In [239]:
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier(random_state=42)

param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],  
    'max_depth': [None, 5, 10, 20],               
    'min_samples_split': [2, 5, 10],            
    'min_samples_leaf': [1, 2, 4],               
    'max_features': [None, 'sqrt', 'log2']      
}

grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

y_pred = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Best Parameters:", best_params)
print("Test Set Accuracy:", accuracy)
print(classification_report(y_test, y_pred_lgb))


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Test Set Accuracy: 0.9016205988576748
              precision    recall  f1-score   support

           0       0.90      0.86      0.88    443490
           1       0.74      0.89      0.81    228276
           2       0.93      0.86      0.90    376797

    accuracy                           0.87   1048563
   macro avg       0.86      0.87      0.86   1048563
weighted avg       0.88      0.87      0.87   1048563



In [240]:
import joblib

joblib.dump(best_estimator, 'best_decision_tree_model.pkl')

# You can later load the model like this:
# loaded_model = joblib.load('best_decision_tree_model.pkl')


['best_decision_tree_model.pkl']